In [1]:
pip install recordlinkage

     |████████████████████████████████| 952kB 8.4MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 


In [2]:
import pandas as pd
import recordlinkage

In [3]:

hospital_accounts = pd.read_csv('hospital_account_info.csv', index_col='Account_Num')
hospital_reimbursement = pd.read_csv('hospital_reimbursement.csv', index_col='Provider_Num')

In [4]:
indexer = recordlinkage.Index()
indexer.full()

<Index>

In [5]:
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidates))

14399283


In [6]:
compare = recordlinkage.Compare()
compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
            'Provider Name',
            threshold=0.85,
            label='Hosp_Name')
compare.string('Address',
            'Provider Street Address',
            method='jarowinkler',
            threshold=0.85,
            label='Hosp_Address')
features = compare.compute(candidates, hospital_accounts,
                        hospital_reimbursement)

KeyboardInterrupt: ignored

In [ ]:
indexer = recordlinkage.Index()
indexer.block(left_on='State', right_on='Provider State')
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidates))

In [ ]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='State', right_on='Provider State')
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidates))

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0        2285
2.0         474
1.0       22262
0.0    14374262
dtype: int64

In [ ]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'City':'Hosp_Address'].sum(axis=1)

In [ ]:
hospital_accounts['Acct_Name_Lookup'] = hospital_accounts[[
    'Facility Name', 'Address', 'City', 'State'
]].apply(lambda x: '_'.join(x), axis=1)

hospital_reimbursement['Reimbursement_Name_Lookup'] = hospital_reimbursement[[
    'Provider Name', 'Provider Street Address', 'Provider City',
    'Provider State'
]].apply(lambda x: '_'.join(x), axis=1)

account_lookup = hospital_accounts[['Acct_Name_Lookup']].reset_index()
reimbursement_lookup = hospital_reimbursement[['Reimbursement_Name_Lookup']].reset_index()

In [ ]:
account_merge = potential_matches.merge(account_lookup, how='left')

In [ ]:
final_merge = account_merge.merge(reimbursement_lookup, how='left')

In [ ]:
cols = ['Account_Num', 'Provider_Num', 'Score',
        'Acct_Name_Lookup', 'Reimbursement_Name_Lookup']
final_merge[cols].sort_values(by=['Account_Num', 'Score'], ascending=False)

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
2384,94995,825914,3.0,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...
1869,94953,819181,3.0,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...
1652,94943,680596,3.0,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...
2244,94923,403151,3.0,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...
958,94887,752284,2.0,NEW YORK-PRESBYTERIAN BROOKLYN METHODIST HOSPI...,NEW YORK METHODIST HOSPITAL_506 SIXTH STREET_B...
...,...,...,...,...,...
430,10165,188247,3.0,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT
1691,10090,212069,3.0,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...
922,10043,140535,3.0,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...
1166,10020,210657,3.0,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...


In [ ]:
final_merge[final_merge['Account_Num']==32725][cols]

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup


In [ ]:
final_merge.sort_values(by=['Account_Num', 'Score'],
                    ascending=False).to_excel('merge_list.xlsx',
                                              index=False)

From Here is Recod Linkage


In [ ]:
hospital_dupes = pd.read_csv('hospital_account_dupes.csv', index_col='Account_Num')

In [ ]:
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='State')
dupe_candidate_links = dupe_indexer.index(hospital_dupes)

In [ ]:
compare_dupes = recordlinkage.Compare()
compare_dupes.string('City', 'City', threshold=0.85, label='City')
compare_dupes.string('Phone Number',
                    'Phone Number',
                    threshold=0.85,
                    label='Phone_Num')
compare_dupes.string('Facility Name',
                    'Facility Name',
                    threshold=0.80,
                    label='Hosp_Name')
compare_dupes.string('Address',
                    'Address',
                    threshold=0.85,
                    label='Hosp_Address')
dupe_features = compare_dupes.compute(dupe_candidate_links, hospital_dupes)

In [ ]:
print(dupe_features)

                             City  Phone_Num  Hosp_Name  Hosp_Address
Account_Num_1 Account_Num_2                                          
26270         28485           0.0        0.0        0.0           0.0
              30430           0.0        0.0        0.0           0.0
              43602           0.0        0.0        0.0           0.0
59585         28485           0.0        0.0        0.0           0.0
              30430           0.0        0.0        0.0           0.0
...                           ...        ...        ...           ...
64029         38600           0.0        0.0        0.0           0.0
              35413           0.0        0.0        0.0           0.0
              81525           0.0        0.0        0.0           0.0
              82916           0.0        0.0        0.0           0.0
              18907           0.0        0.0        0.0           0.0

[981277 rows x 4 columns]


In [ ]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0         7
2.0       206
1.0      7859
0.0    973205
dtype: int64

In [ ]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 1].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'City':'Hosp_Address'].sum(axis=1)

In [ ]:
print(potential_dupes)

     Account_Num_1  Account_Num_2  City  ...  Hosp_Name  Hosp_Address  Score
0            82869          52690   1.0  ...        0.0           1.0    2.0
1            57289          26270   1.0  ...        1.0           0.0    2.0
2            21838          70883   1.0  ...        0.0           0.0    2.0
3            39717          77788   1.0  ...        0.0           1.0    2.0
4            39717          61353   1.0  ...        0.0           0.0    2.0
..             ...            ...   ...  ...        ...           ...    ...
208          78300          80984   1.0  ...        0.0           0.0    2.0
209          74177          50721   1.0  ...        0.0           0.0    2.0
210          65652          54940   1.0  ...        0.0           0.0    2.0
211          65652          71903   1.0  ...        1.0           0.0    2.0
212          89851          77850   1.0  ...        0.0           0.0    2.0

[213 rows x 7 columns]


RECORD LINKAGE IS STARTING FROM HERE


In [85]:
dataset1 = pd.read_csv('hospital_account_info.csv')

In [86]:
dataset2 = pd.read_csv('hospital_reimbursement.csv')

In [87]:
df_a = pd.DataFrame(dataset1)
df_b = pd.DataFrame(dataset2)

In [88]:
indexer = recordlinkage.Index()
indexer.block('Provider_Num', 'Provider_Num')
candidate_links = indexer.index(df_a, df_b)
c = recordlinkage.Compare()


In [74]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(df_a, df_b)
c = recordlinkage.Compare()

In [ ]:
# Comparison step - DO NOT EXECUTE

c.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
c.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
c.exact('suburb', 'suburb', label='suburb')
c.exact('state', 'state', label='state')
c.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA, dfB)

In [89]:
c.string('Facility_Name', 'Provider_Name', method='jarowinkler', threshold=0.85,label='Facility_Name')

<Compare>

In [90]:
c.numeric('ZIP_Code', 'Provider_Zip_Code',label='ZIP_Code')

<Compare>

In [91]:
c.numeric('Provider_Num', 'Provider_Num',label='Provider_Num')

<Compare>

In [92]:
c.exact('State', 'Provider_State', label='state')

<Compare>

didn't run below two 46 an 45


In [93]:
feature_vectors = c.compute(candidate_links, df_a, df_b)

In [94]:
print(feature_vectors)

         Facility_Name  ZIP_Code  Provider_Num  state
0   5              0.0       0.0           1.0      0
1   6              0.0       0.0           1.0      0
2   7              0.0       0.0           1.0      0
3   8              0.0       0.0           1.0      0
4   9              0.0       0.0           1.0      0
...                ...       ...           ...    ...
712 717            0.0       0.0           1.0      0
713 718            0.0       0.0           1.0      0
714 719            0.0       0.0           1.0      0
715 720            0.0       0.0           1.0      0
716 721            0.0       0.0           1.0      0

[717 rows x 4 columns]


In [95]:
ecm = recordlinkage.ECMClassifier()


In [96]:
ecm.fit_predict(feature_vectors)

MultiIndex([(321, 326)],
           )

In [101]:
df_a.loc[321]

Provider_Num                                             345997
Facility_Name         KAISER FOUNDATION HOSPITAL - REDWOOD CITY
Address                                 1100 VETERANS BOULEVARD
City                                               REDWOOD CITY
State                                                        CA
ZIP_Code                                                  94063
County_Name                                           SAN MATEO
Phone_Number                                     (650) 299-2000
Hospital_Type                              Acute Care Hospitals
Hospital_Ownership               Voluntary non-profit - Private
Name: 321, dtype: object

In [102]:
df_b.loc[326]

Provider_Num                                                 345997
Provider_Name                KAISER FOUNDATION HOSPITAL - SAN DIEGO
Address                                               4647 ZION AVE
Provider_City                                             SAN DIEGO
Provider_State                                                   CA
Provider_Zip_Code                                             92120
Total_Discharges                                                 29
Average_Covered_Charges                                     35732.7
Average_Total_Payments                                      10634.4
Average_Medicare_Payments                                    5235.9
Name: 326, dtype: object